# Multi-layer Perceptron (MLP)

## 외부 파일 가져오기 & requirements 설치

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import os
import sys
sys.path.append("/content/drive/MyDrive/#fastcampus")
!pip install -r "/content/drive/MyDrive/#fastcampus/requirements.txt"

In [8]:
import os
import sys

In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import random_split

from torchvision.datasets import FashionMNIST
from torchvision import transforms

In [5]:
pwd

'C:\\Users\\User\\LGUplus_Fastcampus\\한번에끝내는딥러닝\\Part4_pytorch'

In [6]:
from data_utils import dataset_split

In [9]:
data_root = os.path.join(os.getcwd(), "data")

# 전처리 부분 (preprocessing) & 데이터 셋 정의.
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]), # mean, # std
    ]
)
fashion_mnist_dataset = FashionMNIST(data_root, download=True, train=True, transform=transform)


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting C:\Users\User\LGUplus_Fastcampus\한번에끝내는딥러닝\Part4_pytorch\data\FashionMNIST\raw\train-images-idx3-ubyte.gz to C:\Users\User\LGUplus_Fastcampus\한번에끝내는딥러닝\Part4_pytorch\data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting C:\Users\User\LGUplus_Fastcampus\한번에끝내는딥러닝\Part4_pytorch\data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to C:\Users\User\LGUplus_Fastcampus\한번에끝내는딥러닝\Part4_pytorch\data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting C:\Users\User\LGUplus_Fastcampus\한번에끝내는딥러닝\Part4_pytorch\data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\User\LGUplus_Fastcampus\한번에끝내는딥러닝\Part4_pytorch\data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting C:\Users\User\LGUplus_Fastcampus\한번에끝내는딥러닝\Part4_pytorch\data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\User\LGUplus_Fastcampus\한번에끝내는딥러닝\Part4_pytorch\data\FashionMNIST\raw



## Dataloader를 정의

In [10]:
datasets = dataset_split(fashion_mnist_dataset, split=[0.9, 0.1])

train_dataset = datasets["train"]
val_dataset = datasets["val"]

train_batch_size = 100
val_batch_size = 10

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=1
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_batch_size, shuffle=False, num_workers=1
)


In [11]:
for sample_batch in train_dataloader:
    print(sample_batch)
    print(sample_batch[0].shape, sample_batch[1].shape)
    break

[tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],


        [[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],


        [[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.00

## 모델 (Multi-layer Perceptron) (MLP) ! 정의

In [12]:
# Define Model.

class MLP(nn.Module):
    def __init__(self, in_dim: int, h1_dim: int, h2_dim: int, out_dim: int):
        super().__init__()
        self.linear1 = nn.Linear(in_dim, h1_dim)
        self.linear2 = nn.Linear(h1_dim, h2_dim)
        self.linear3 = nn.Linear(h2_dim, out_dim)
        self.relu = F.relu
        
    def forward(self, input):
        x = torch.flatten(input, start_dim=1)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        out = self.linear3(x)
        # out = F.softmax(out)
        return out

## 모델 선언 및 손실 함수, 최적화(Optimizer) 정의, Tensorboard Logger 정의 

In [13]:
# define model.
model = MLP(28*28, 128, 64, 10)

# define loss
loss_function = nn.CrossEntropyLoss()

# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
max_epoch = 15

# define tensorboard logger
writer = SummaryWriter()
log_interval = 100

In [14]:
%load_ext tensorboard
%tensorboard --logdir runs/

train_step = 0
for epoch in range(1, max_epoch+1):
    # valid step
    with torch.no_grad():
        val_loss = 0.0
        val_corrects = 0

        for val_batch_idx, (val_images, val_labels) in enumerate(
            tqdm(val_dataloader, position=0, leave=True, desc="validation")
        ):
            # forward
            val_outputs = model(val_images)
            _, val_preds = torch.max(val_outputs, 1)
            
            # loss & acc
            val_loss += loss_function(val_outputs, val_labels) / val_outputs.shape[0]
            val_corrects += torch.sum(val_preds == val_labels.data) / val_outputs.shape[0]
    
    # valid step logging
    val_epoch_loss = val_loss / len(val_dataloader)
    val_epoch_acc = val_corrects / len(val_dataloader)
    
    print(
        f"{epoch} epoch, {train_step} step: val_loss: {val_epoch_loss}, val_acc: {val_epoch_acc}"
    )
    writer.add_scalar("Loss/val", val_epoch_loss, train_step)
    writer.add_scalar("Acc/val", val_epoch_acc, train_step)
    writer.add_images("Images/val", val_images, train_step)
    
    # train step
    current_loss = 0
    current_corrects = 0

    for batch_idx, (images, labels) in enumerate(
         tqdm(train_dataloader, position=0, leave=True, desc="training")
    ):
        current_loss = 0.0
        current_corrects = 0

        # Forward
        # get predictions
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        # get loss (Loss 계산)
        loss = loss_function(outputs, labels)

        # Backpropagation
        # optimizer 초기화 (zero화)
        optimizer.zero_grad()

        # Perform backward pass
        loss.backward()

        # Perform Optimization
        optimizer.step()

        current_loss += loss.item()
        current_corrects += torch.sum(preds == labels.data)

        if train_step % log_interval == 0:
            train_loss = current_loss / log_interval
            train_acc = current_corrects / log_interval

            print(
                f"{train_step}: train_loss: {train_loss}, train_acc: {train_acc}"
            )
            writer.add_scalar("Loss/train", train_loss, train_step)
            writer.add_scalar("Acc/train", train_acc, train_step)
            writer.add_images("Images/train", images, train_step)
            writer.add_graph(model, images)
            current_loss = 0
            current_corrects = 0

        train_step += 1

validation: 100%|███████████████████████████████████████████████████████████████████| 600/600 [00:04<00:00, 142.11it/s]


1 epoch, 0 step: val_loss: 0.232170969247818, val_acc: 0.12566645443439484


training:   0%|                                                                                | 0/540 [00:00<?, ?it/s]

0: train_loss: 0.023142566680908205, train_acc: 0.10999999940395355


training:  20%|█████████████▊                                                        | 107/540 [00:04<00:10, 40.19it/s]

...

training:  64%|█████████████████████████████████████████████                         | 348/540 [00:08<00:04, 47.87it/s]

7900: train_loss: 0.0015615320205688476, train_acc: 0.9300000071525574


training:  82%|█████████████████████████████████████████████████████████▌            | 444/540 [00:10<00:02, 45.52it/s]

8000: train_loss: 0.002220032662153244, train_acc: 0.9100000262260437


training: 100%|██████████████████████████████████████████████████████████████████████| 540/540 [00:12<00:00, 43.00it/s]


In [ ]:
# save model
os.makedirs("./logs/models", exist_ok=True)
torch.save(model, "./logs/models/mlp.ckpt")

In [ ]:
# load model
loaded_model = torch.load("./logs/models/mlp.ckpt")
loaded_model.eval()
print(loaded_model)

In [ ]:
def softmax(x, axis=0):
    "numpy softmax"
    max = np.max(x, axis=axis, keepdims=True)
    e_x = np.exp(x - max)
    sum = np.sum(e_x, axis=axis, keepdims=True)
    f_x = e_x / sum
    return f_x

In [ ]:
test_batch_size = 100
test_dataset = FashionMNIST(data_root, download=True, train=False, transform=transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=1)

test_labels_list = []
test_preds_list = []
test_outputs_list = []
for i, (test_images, test_labels) in enumerate(tqdm(test_dataloader, position=0, leave=True, desc="testing")):
    # forward
    test_outputs = loaded_model(test_images)
    _, test_preds = torch.max(test_outputs, 1)

    final_outs = softmax(test_outputs.detach().numpy(), axis=1)
    test_outputs_list.extend(final_outs)
    test_preds_list.extend(test_preds.detach().numpy())
    test_labels_list.extend(test_labels.detach().numpy())

test_preds_list = np.array(test_preds_list)
test_labels_list = np.array(test_labels_list)

print(f"\nacc: {np.mean(test_preds_list == test_labels_list)*100}%")

In [ ]:
# ROC Curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr = {}
tpr = {}
thresh = {}
n_class = 10

for i in range(n_class):
    fpr[i], tpr[i], thresh[i] = roc_curve(test_labels_list, np.array(test_outputs_list)[:, i], pos_label=i)

# plot.
for i in range(n_class):
    plt.plot(fpr[i], tpr[i], linestyle="--", label=f"Class {i} vs Rest")
plt.title("Multi-class ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="best")
plt.show()

print("auc_score", roc_auc_score(test_labels_list, test_outputs_list, multi_class="ovo", average="macro"))